In [1]:
import base64
import datetime
import io

import dash
from dash.dependencies import Input, Output, State
import dash_core_components as dcc
import dash_html_components as html
import dash_table

import pandas as pd
import json
import os
import re
import urllib

In [ ]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']







app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div([
    html.Div([html.Img(src="https://media-exp1.licdn.com/dms/image/C560BAQFX4cUstccvPg/company-logo_200_200/0/1519929513217?e=1625097600&v=beta&t=qvlGh7xpTzIpqv50e7u4V7lJP8nflgLGo3DJxtuW1eQ",
                             style={'width':'10%'
                                   }),
                    html.H2('Upload App',
                            style={'margin-top': '0%',
                                                   'padding-top':'0px'}
                           ),
             html.Div([html.A("Developed By Alex Surin", href='https://www.linkedin.com/in/alexandr-surin-59360773/', target="_blank")]
                     )],
                    style={'textAlign': 'center'
                          }),
    dcc.Upload(
        id='upload-data',
        children=html.Div([
            'Drag and Drop or ',
            html.A('Select Files')
        ]),
        style={
            'width': '100%',
            'height': '60px',
            'lineHeight': '60px',
            'borderWidth': '1px',
            'borderStyle': 'dashed',
            'borderRadius': '5px',
            'textAlign': 'center',
            'margin': '10px'
        },
        # Allow multiple files to be uploaded
        multiple=False
    ),
    
    html.Div([html.Label('Select Delimiter',style={
                                      'margin-top': '15px',
                                      'padding': '5px'
                                     }), 
                    dcc.RadioItems(id='user_format_input',
                                   options=[
                                       {'label': 'Tab', 'value':'tsv'},
                                       {'label': 'Comma', 'value': 'csv'}
                                   ],
                                   value='csv',
                                       labelStyle={'display': 'inline-block'}
                                  )
             ],style={'textAlign': 'center'
                          }),
    html.Div(id='output-data-upload')
])

        
def parse_file(contents, filename, date,user_format_input):
    def fix_string(df):
        for col in (df.select_dtypes(exclude='int64').columns):
            #print(col)
            #Fix triple
            df[col]=df[[col]].applymap(lambda x:x.encode().decode('utf-8').replace('\u201c\u201d\u201d', '\"').replace('\u201d\u201d\u201d', '\"'))
            #Fix single
            df[col]=df[[col]].applymap(lambda x:x.encode().decode('utf-8').replace('\u201c', '\"').replace('\u201d', '\"'))
        return df
    try:
        content_type, content_string = contents.split(',')
        print('Content type: {}'.format(content_type))
        print('Content string: {}'.format(content_string))
        decoded = base64.b64decode(content_string)
        print('Decoded: {}'.format(decoded))
        print('USER INPUT: {}'.format(user_format_input))
        print('BOOL:{}'.format(bool(re.search('text/csv', content_string))))
        if user_format_input =='csv' and bool(re.search('text/csv', content_type)):
            df = pd.read_csv(io.StringIO(decoded.decode('utf-8')))
        elif user_format_input =='tsv'and bool(re.search('text/tab', content_type)):
            df = pd.read_csv(io.StringIO(decoded.decode('utf-8')), sep='\t')
        else:return html.Div([
            'Wrong Delimiter Selected! Try again.'
        ],style={'width':'49%',
                            'background-color': '#ff7f7f',
                            'margin': '15px',
                            'padding': '5px',
                            'border':'1px solid #ff7f7f',
                            'border-radius': '10px',
                            'display': 'inline-block'
                                       })        
            
            
            
        #Fix stings
        df_pd=fix_string(df)
        #Get JSON
        df_json=json.dumps(df.to_dict(orient="records"))
        #####
        #Download
        ###
        json_string="data:text;charset=utf-8," + urllib.parse.quote(json.dumps(df.to_dict(orient="records")).encode('utf-8'))
        ####
        return html.Div([
            html.H5('Uploaded File: {}'.format(filename)),
            html.Div([#Display Table
                html.Label('Table Preview',
                                style={
                                    'margin-bottom': '15px',
                                    'margin-top': '15px',
                                     }),
                dash_table.DataTable(
                    data=df_pd.to_dict('records'),
                    columns=[{'name': i, 'id': i} for i in df_pd.columns]
                    ),
                html.Hr(),
                html.Div(#Display JSON
                    
                    [html.Label('JSON Output',
                                style={
                                    'margin-bottom': '15px',
                                    'margin-top': '15px',
                                     }),
                     '{}'.format(df_json),
                    html.Hr(),
                    html.Label('Download JSON',
                                style={
                                    'margin-bottom': '15px',
                                    'margin-top': '15px',
                                     }),
                    html.A(
                        'Download',
                        id='download-link',
                        download="rawjson_download.json",
                        href=json_string,
                        #target="_blank"
                    )]),
            ],style={'width':'49%',
                            'background-color': '#F2F2F2',
                            'margin': '15px',
                            'padding': '5px',
                            'border':'1px solid #F2F2F2',
                            'border-radius': '10px',
                            'display': 'inline-block'
                                       })],style={'width':'49%',
                            'background-color': '#F9F9F9',
                            'margin': '15px',
                            'padding': '5px',
                            'border':'1px solid ##F9F9F9',
                            'border-radius': '10px',
                            'display': 'inline-block'
                                       })
    
        
    except Exception as e:
        print(e)
        return html.Div([
            'There was an error processing this file.'
        ],style={'width':'49%',
                            'background-color': '#ff7f7f',
                            'margin': '15px',
                            'padding': '5px',
                            'border':'1px solid #ff7f7f',
                            'border-radius': '10px',
                            'display': 'inline-block'
                                       })        
@app.callback(Output('output-data-upload', 'children'),
              [Input('upload-data', 'contents'),
               Input('upload-data', 'filename'),
               Input('upload-data', 'last_modified'),
               Input('user_format_input', 'value')]
             )        

def update_output(list_of_contents, list_of_names, list_of_dates,user_format_input):
    if list_of_contents is not None:
        children = [
            parse_file(list_of_contents, list_of_names,list_of_dates,user_format_input)]
        return children



if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 in production, use a production WSGI server like gunicorn instead.

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [07/Apr/2021 15:05:34] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/Apr/2021 15:05:36] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [07/Apr/2021 15:05:36] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [07/Apr/2021 15:05:36] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [07/Apr/2021 15:05:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [07/Apr/2021 15:05:45] "GET /_dash-component-suites/dash_table/async-table.v4_8_1m1592577109.js HTTP/1.1" 200 -


Content type: data:text/csv;base64
Content string: SUQsTmFtZSxBZ2UsRmF2b3JpdGUgQ29sb3INCjEsQm9iLDIzLFJlZA0KMiwi4oCcRG9lLEpvaG7igJ0iLDk5LOKAnOKAneKAnVR1cnF1b2lzZeKAneKAneKAnQ==
Decoded: b'ID,Name,Age,Favorite Color\r\n1,Bob,23,Red\r\n2,"\xe2\x80\x9cDoe,John\xe2\x80\x9d",99,\xe2\x80\x9c\xe2\x80\x9d\xe2\x80\x9dTurquoise\xe2\x80\x9d\xe2\x80\x9d\xe2\x80\x9d'
USER INPUT: csv
BOOL:False


127.0.0.1 - - [07/Apr/2021 15:05:55] "POST /_dash-update-component HTTP/1.1" 200 -


Content type: data:text/csv;base64
Content string: SUQsTmFtZSxBZ2UsRmF2b3JpdGUgQ29sb3INCjEsQm9iLDIzLFJlZA0KMiwi4oCcRG9lLEpvaG7igJ0iLDk5LOKAnOKAneKAnVR1cnF1b2lzZeKAneKAneKAnQ==
Decoded: b'ID,Name,Age,Favorite Color\r\n1,Bob,23,Red\r\n2,"\xe2\x80\x9cDoe,John\xe2\x80\x9d",99,\xe2\x80\x9c\xe2\x80\x9d\xe2\x80\x9dTurquoise\xe2\x80\x9d\xe2\x80\x9d\xe2\x80\x9d'
USER INPUT: tsv
BOOL:False
